In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tqdm

In [2]:
# Read data
pre = 'norm_fog_data/'
tasks = ['1','2','3','4','5','6','7','8','9']
prefixes = ['001', '002', '003', '004', '005', '006', '007', '008', '009','010','011','012']
filenames = []
for prefix in prefixes:
    if prefix not in ['004','008','009']:
        for task in tasks[:4]:
            file = pre + prefix + '_task_' + task + '.csv'
            filenames.append(file)      
    elif prefix == '004':
        for task in tasks[:5]:
            file = pre + prefix + '_task_' + task + '.csv'
            filenames.append(file)
    elif prefix == '008':
        for task in tasks:
            file = pre + prefix + '_task_' + task + '.csv'
            filenames.append(file)
    else:
        for task in tasks[:6]:
            file = pre + prefix + '_task_' + task + '.csv'
            filenames.append(file)

print(filenames)

['norm_fog_data/001_task_1.csv', 'norm_fog_data/001_task_2.csv', 'norm_fog_data/001_task_3.csv', 'norm_fog_data/001_task_4.csv', 'norm_fog_data/002_task_1.csv', 'norm_fog_data/002_task_2.csv', 'norm_fog_data/002_task_3.csv', 'norm_fog_data/002_task_4.csv', 'norm_fog_data/003_task_1.csv', 'norm_fog_data/003_task_2.csv', 'norm_fog_data/003_task_3.csv', 'norm_fog_data/003_task_4.csv', 'norm_fog_data/004_task_1.csv', 'norm_fog_data/004_task_2.csv', 'norm_fog_data/004_task_3.csv', 'norm_fog_data/004_task_4.csv', 'norm_fog_data/004_task_5.csv', 'norm_fog_data/005_task_1.csv', 'norm_fog_data/005_task_2.csv', 'norm_fog_data/005_task_3.csv', 'norm_fog_data/005_task_4.csv', 'norm_fog_data/006_task_1.csv', 'norm_fog_data/006_task_2.csv', 'norm_fog_data/006_task_3.csv', 'norm_fog_data/006_task_4.csv', 'norm_fog_data/007_task_1.csv', 'norm_fog_data/007_task_2.csv', 'norm_fog_data/007_task_3.csv', 'norm_fog_data/007_task_4.csv', 'norm_fog_data/008_task_1.csv', 'norm_fog_data/008_task_2.csv', 'norm_f

In [20]:
import numpy as np
# Use patient 1 and 5 as test set
# Start with window size of 1s
train = np.empty((10677,500,32))
test = np.empty((1745,500,32))
train_counter = 0
test_counter = 0
train_1s_windows = 0
test_1s_windows = 0
for file in filenames:
    if file[14:17] not in ['001','005','007']:
        print('Adding ', file, ' to training set')
        df = pd.read_csv(file)
        data = np.array(df)
        data = data[:data.shape[0]-1,:]
        data = data.reshape((-1,500,32))
        train[train_counter:train_counter+data.shape[0]] = data
        train_counter += data.shape[0]
        length = data.shape[0]
        train_1s_windows += int((length))
    else:
        print('Adding ', file, ' to test set')
        df = pd.read_csv(file)
        data = np.array(df)
        data = data[:data.shape[0]-1,:]
        data = data.reshape((-1,500,32))
        test[test_counter:test_counter+data.shape[0]] = data
        test_counter += data.shape[0]
        length = data.shape[0]
        test_1s_windows += int((length))

print(train_1s_windows, test_1s_windows)

Adding  norm_fog_data/001_task_1.csv  to test set
Adding  norm_fog_data/001_task_2.csv  to test set
Adding  norm_fog_data/001_task_3.csv  to test set
Adding  norm_fog_data/001_task_4.csv  to test set
Adding  norm_fog_data/002_task_1.csv  to training set
Adding  norm_fog_data/002_task_2.csv  to training set
Adding  norm_fog_data/002_task_3.csv  to training set
Adding  norm_fog_data/002_task_4.csv  to training set
Adding  norm_fog_data/003_task_1.csv  to training set
Adding  norm_fog_data/003_task_2.csv  to training set
Adding  norm_fog_data/003_task_3.csv  to training set
Adding  norm_fog_data/003_task_4.csv  to training set
Adding  norm_fog_data/004_task_1.csv  to training set
Adding  norm_fog_data/004_task_2.csv  to training set
Adding  norm_fog_data/004_task_3.csv  to training set
Adding  norm_fog_data/004_task_4.csv  to training set
Adding  norm_fog_data/004_task_5.csv  to training set
Adding  norm_fog_data/005_task_1.csv  to test set
Adding  norm_fog_data/005_task_2.csv  to test se

In [7]:
np.random.seed(0)
np.random.shuffle(train)
np.random.shuffle(test)

In [8]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim):
        super(RNNModel, self).__init__()

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(fc_dim, output_dim)

    def forward(self, x):

        out, h0 = self.rnn(x)

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out.flatten(start_dim=-2))
        return torch.sigmoid(out)

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim):
        super(LSTMModel, self).__init__()

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(fc_dim, output_dim)

    def forward(self, x):

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x)

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out.flatten(start_dim=-2))

        return torch.sigmoid(out)

In [10]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim):
        super(GRUModel, self).__init__()

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(fc_dim, output_dim)

    def forward(self, x):
        
        out, _ = self.gru(x)

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out.flatten(start_dim=-2))

        return torch.sigmoid(out)

In [13]:
# Vanilla RNN 1
input_dim = 30
hidden_dim = 1 
layer_dim = 2 
output_dim = 1 
dropout_prob = 0
fc_dim = 500
rnn = RNNModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(rnn.parameters(),lr=1e-2,weight_decay=0)
rnn.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        rnn.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = rnn(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(rnn, 'train loss')
        train_accuracy = test_model(rnn, 'train accuracy')
        test_accuracy = test_model(rnn, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.6863, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.5576)
Test Accuracy:  tensor(0.6602)
Starting epoch:  1
Training loss:  tensor(0.6661, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.5978)
Test Accuracy:  tensor(0.6510)
Starting epoch:  2
Training loss:  tensor(0.5702, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7264)
Test Accuracy:  tensor(0.7060)
Starting epoch:  3
Training loss:  tensor(0.4833, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7750)
Test Accuracy:  tensor(0.7278)
Starting epoch:  4
Training loss:  tensor(0.4704, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7841)
Test Accuracy:  tensor(0.7318)


In [19]:
# Vanilla RNN 1
input_dim = 30
hidden_dim = 1 
layer_dim = 3 
output_dim = 1 
dropout_prob = 0
fc_dim = 500
rnn = RNNModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(rnn.parameters(),lr=1e-2,weight_decay=0)
rnn.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        rnn.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = rnn(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(rnn, 'train loss')
        train_accuracy = test_model(rnn, 'train accuracy')
        test_accuracy = test_model(rnn, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.5782, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7093)
Test Accuracy:  tensor(0.6974)
Starting epoch:  1
Training loss:  tensor(0.5014, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7672)
Test Accuracy:  tensor(0.7186)
Starting epoch:  2
Training loss:  tensor(0.4798, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7751)
Test Accuracy:  tensor(0.7129)
Starting epoch:  3
Training loss:  tensor(0.4840, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7757)
Test Accuracy:  tensor(0.7307)
Starting epoch:  4
Training loss:  tensor(0.4778, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7783)
Test Accuracy:  tensor(0.7192)


Vanilla RNN 1 Results: training loss - 0.4704, training accuracy - 0.7841, test accuracy - 0.7318

In [15]:
# LSTM stacked
input_dim = 30
hidden_dim = 1 
layer_dim = 2 
output_dim = 1 
dropout_prob = 0
fc_dim = 500
lstm = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(lstm.parameters(),lr=1e-2,weight_decay=0)
lstm.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        lstm.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = lstm(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(lstm, 'train loss')
        train_accuracy = test_model(lstm, 'train accuracy')
        test_accuracy = test_model(lstm, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.5151, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7632)
Test Accuracy:  tensor(0.6355)
Starting epoch:  1
Training loss:  tensor(0.3930, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8304)
Test Accuracy:  tensor(0.6567)
Starting epoch:  2
Training loss:  tensor(0.3781, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8389)
Test Accuracy:  tensor(0.6653)
Starting epoch:  3
Training loss:  tensor(0.3696, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8424)
Test Accuracy:  tensor(0.6797)
Starting epoch:  4
Training loss:  tensor(0.3655, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8462)
Test Accuracy:  tensor(0.6894)


In [14]:
# LSTM stacked
input_dim = 30
hidden_dim = 1 
layer_dim = 3
output_dim = 1 
dropout_prob = 0
fc_dim = 500
lstm = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(lstm.parameters(),lr=1e-2,weight_decay=0)
lstm.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        lstm.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = lstm(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(lstm, 'train loss')
        train_accuracy = test_model(lstm, 'train accuracy')
        test_accuracy = test_model(lstm, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.4719, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8018)
Test Accuracy:  tensor(0.7255)
Starting epoch:  1
Training loss:  tensor(0.4019, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8299)
Test Accuracy:  tensor(0.7330)
Starting epoch:  2
Training loss:  tensor(0.3861, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8362)
Test Accuracy:  tensor(0.7192)
Starting epoch:  3
Training loss:  tensor(0.3760, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8426)
Test Accuracy:  tensor(0.7060)
Starting epoch:  4
Training loss:  tensor(0.3699, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8451)
Test Accuracy:  tensor(0.7089)


In [16]:
# LSTM not stacked
input_dim = 30
hidden_dim = 1 
layer_dim = 1 
output_dim = 1 
dropout_prob = 0
fc_dim = 500
lstm = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(lstm.parameters(),lr=1e-2,weight_decay=0)
lstm.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        lstm.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = lstm(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(lstm, 'train loss')
        train_accuracy = test_model(lstm, 'train accuracy')
        test_accuracy = test_model(lstm, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.5898, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7058)
Test Accuracy:  tensor(0.6052)
Starting epoch:  1
Training loss:  tensor(0.4449, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7966)
Test Accuracy:  tensor(0.7146)
Starting epoch:  2
Training loss:  tensor(0.3992, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8216)
Test Accuracy:  tensor(0.7181)
Starting epoch:  3
Training loss:  tensor(0.3870, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8267)
Test Accuracy:  tensor(0.7278)
Starting epoch:  4
Training loss:  tensor(0.3797, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8296)
Test Accuracy:  tensor(0.7358)


In [17]:
# GRU
input_dim = 30
hidden_dim = 1 
layer_dim = 1 
output_dim = 1 
dropout_prob = 0
fc_dim = 500
gru = GRUModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(gru.parameters(),lr=1e-2,weight_decay=0)
gru.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        gru.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = gru(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(gru, 'train loss')
        train_accuracy = test_model(gru, 'train accuracy')
        test_accuracy = test_model(gru, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.7133, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.5397)
Test Accuracy:  tensor(0.5673)
Starting epoch:  1
Training loss:  tensor(0.7019, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.5958)
Test Accuracy:  tensor(0.6029)
Starting epoch:  2
Training loss:  tensor(0.5127, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7721)
Test Accuracy:  tensor(0.4854)
Starting epoch:  3
Training loss:  tensor(0.4528, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7964)
Test Accuracy:  tensor(0.5524)
Starting epoch:  4
Training loss:  tensor(0.4371, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8043)
Test Accuracy:  tensor(0.5885)


In [18]:
# GRU
input_dim = 30
hidden_dim = 1 
layer_dim = 2 
output_dim = 1 
dropout_prob = 0
fc_dim = 500
gru = GRUModel(input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, fc_dim)
optimizer = torch.optim.Adam(gru.parameters(),lr=1e-2,weight_decay=0)
gru.double()

criterion = nn.BCELoss()
train_loss = []
test_loss = []
# Train
with torch.autograd.set_detect_anomaly(True):
    for epoch in range(5):
        gru.train()
        print('Starting epoch: ', epoch)
        running_loss = 0
        for i in range(0, train.shape[0], 64):
            optimizer.zero_grad()
            data = train[i:i+64]
            y = data[:,:,1]
            y[y<0] = 0
            y[y>0] = 1
            y = y.mean(axis=1)
            y = np.round_(y)
            y = torch.tensor(y)
            input = torch.tensor(data[:,:,2:])
            output = gru(input)
            loss = criterion(output.flatten(), y)
            loss.backward()     
            optimizer.step()
            
        running_loss = test_model(gru, 'train loss')
        train_accuracy = test_model(gru, 'train accuracy')
        test_accuracy = test_model(gru, 'test accuracy')
        train_loss.append(train_accuracy)
        test_loss.append(test_accuracy)
        print('Training loss: ', running_loss)
        print('Training Accuracy: ', train_accuracy)
        print('Test Accuracy: ', test_accuracy)
        

Starting epoch:  0
Training loss:  tensor(0.6744, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.6124)
Test Accuracy:  tensor(0.6579)
Starting epoch:  1
Training loss:  tensor(0.4847, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.7867)
Test Accuracy:  tensor(0.6023)
Starting epoch:  2
Training loss:  tensor(0.4486, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8034)
Test Accuracy:  tensor(0.6206)
Starting epoch:  3
Training loss:  tensor(0.4291, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8127)
Test Accuracy:  tensor(0.6275)
Starting epoch:  4
Training loss:  tensor(0.4159, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)
Training Accuracy:  tensor(0.8196)
Test Accuracy:  tensor(0.6424)


In [12]:
def test_model(model, mode):
    model.eval()
    if mode == 'test accuracy':
        out = model(torch.tensor(test[:,:,2:]))
        out = out.flatten()
        y = test[:,:,1]
        y[y<0] = 0
        y[y>0] = 1
        y = y.mean(axis=1)
        y = np.round_(y)
        y = torch.tensor(y)
        accuracy = torch.sum(torch.round(out.flatten()) == y)/y.shape[0]
        return accuracy
    elif mode == 'train accuracy':
        out = model(torch.tensor(train[:,:,2:]))
        out = out.flatten()
        y = train[:,:,1]
        y[y<0] = 0
        y[y>0] = 1
        y = y.mean(axis=1)
        y = np.round_(y)
        y = torch.tensor(y)
        accuracy = torch.sum(torch.round(out.flatten()) == y)/y.shape[0]
        return accuracy
    elif mode == 'train loss':
        criterion = nn.BCELoss()
        out = model(torch.tensor(train[:,:,2:]))
        out = out.flatten()
        y = train[:,:,1]
        y[y<0] = 0
        y[y>0] = 1
        y = y.mean(axis=1)
        y = np.round_(y)
        y = torch.tensor(y)
        return criterion(out.flatten(), y)

Ideas: try CNN-RNN architecture of CNN-LSTM architecture, deeper FC architecture, try regularization methods, try transfer learning by splitting our dataset, data augmentation, larger windows

Do we want to detect or predict FOG?
In patient or out patient prediction (do we assume we have some or no data of the patient FOG to be predicted)?